## Import des librairies

In [ ]:
import pandas as pd 
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from pickle import dump

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Chargement des données

In [ ]:
data = pd.read_csv("../data/raw/nba_logreg.csv")

In [ ]:
data.head()

In [ ]:
sample = data.drop(columns=['TARGET_5Yrs']).head()
sample.to_csv("../data/test/sample.csv", index=False)

In [ ]:
print(f'Le jeu de données contient {data.shape[0]} lignes et {data.shape[1]} colonnes.')

In [ ]:
data.dtypes

La variable `Name` n'est pas une variable pertinente. Pour commencer, on va donc la supprimer. 

In [ ]:
data = data.drop(columns=['Name'])
data.head()

In [ ]:
fig = px.histogram(data, x="PTS" , color="TARGET_5Yrs")
fig.show()

In [ ]:
fig = px.pie(data, names='TARGET_5Yrs', title='Distribution of the target')
fig.show()

Le dataset n'est pas totalement équilibré mais il n'y a pas un grand déséquilibre des classes. 

## Gestion des valeurs manquantes 

In [ ]:
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_data = pd.DataFrame({'column_name': data.columns,
                                 'percent_missing': percent_missing})

In [ ]:
missing_value_data

Toutes les variables ne contiennent pas de valeurs manquantes sauf la variable `3P%` qui contient 0.82% de valeurs manquantes. La stratégie de la moyenne me paraît la plus simple et la plus pertinente. 

In [ ]:
data['3P%'] = data['3P%'].fillna(data['3P%'].mean())

In [ ]:
data.head()

## Réduction de la dimension

On calcule la matrice de corrélation et on supprime les variables fortement corrélées : 

In [ ]:
corrM = data.corr()
fig = px.imshow(corrM, text_auto=True, aspect="auto")
fig.show()

On remarque que les coefficients de corrélation entre la variable `MIN` et les variables `PTS`, `FGM`, `FGA` dépassent 0.9. On va donc supprimer ces trois dernières variables. 

In [ ]:
data = data.drop(columns=['PTS', 'FGM', 'FGA'])
data.head()

In [ ]:
data.shape

In [ ]:
corrM = data.corr()
fig = px.imshow(corrM, text_auto=True, aspect="auto")
fig.show()

La variable `FTM` est fortement corrélée avec la variable `FTA`. Il faut donc en supprimer une. C'est également le cas pour les deux variables `3P Made` et `3PA` et les deux variables `REB` et `OREB`. 

In [ ]:
data = data.drop(columns=['FTA', '3PA', 'OREB'])
data.head()

## Standardisation des variables numériques

In [ ]:
X = data.drop(columns=['TARGET_5Yrs'])
numeric_columns = X.select_dtypes(['int64', 'float64']).columns

In [ ]:
scaler = StandardScaler()
scaler.fit(data[numeric_columns])

In [ ]:
data[numeric_columns] = scaler.transform(data[numeric_columns])

In [ ]:
data.head()

In [ ]:
dump(scaler, open('../models/scaler.pkl', 'wb'))

In [ ]:
data.to_csv("../data/processed/processed_data.csv", index=False)